In [1]:
from tqdm import tqdm
import pandas as pd
import json
import os
import ast
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

import wandb
api = wandb.Api()

In [7]:
project_path = "dri-ice/Composable_Interventions"
# filter_dict = { "state": "finished" }
filter_dict = { "state": "Failed" }

data_frames = []
runs = api.runs(project_path, filters=filter_dict)

# Iterate over eachrun and capture the c        onfig and summary metrics
for run in tqdm(runs, desc=project_path):
    try:
        if "ga_llama3_hparam_search" not in run.config["tag"]:
            continue

        has_summary = {k: v for k, v in run.summary_metrics.items() if k != "_wandb"} != {}
        has_summary = len(run.summary_metrics) > 1
        if has_summary:
            continue
        
        config_frame = pd.DataFrame([run.config])
        summary_frame = pd.DataFrame([run.summary_metrics])
        summary_frame["host"] = run.metadata["host"]
        combined_frame = pd.concat([config_frame, summary_frame], axis=1)
        data_frames.append(combined_frame)
    except Exception as e:
        print(f"Error processing run {run.id}: {e}")

# Sort by 'tag' and '_timestamp' in descending order to have the most recent run first
all_runs_df = pd.concat(data_frames, ignore_index=True)
all_runs_df["interventions"] = all_runs_df["interventions"].astype(str)

all_runs_df = all_runs_df[all_runs_df["qa_question_count_limit"].isnull()]
all_runs_df = all_runs_df[all_runs_df["qa_question_count_limit"].isnull()]
all_runs_df

dri-ice/Composable_Interventions: 100%|██████████| 2162/2162 [00:00<00:00, 2505.61it/s]


,tag,edit,save,seed,dtype,ga_lr,wandb,wbits,device,method,...,number_of_edits,ga_forget_corpora,ga_retain_corpora,compression_dataset,ga_test_sample_size,rmu_max_num_batches,ga_train_sample_size,qa_question_count_limit,_wandb,host
0,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.001000,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,10,None,{'runtime': 10},udc-an37-1
1,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.001000,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,25,None,{'runtime': 10},udc-an37-1
2,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.001000,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,50,None,{'runtime': 10},udc-an37-1
3,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.001000,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,100,None,{'runtime': 23},udc-an37-1
4,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.001000,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,200,None,{'runtime': 10},udc-an37-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.000050,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,400,None,{'runtime': 10},udc-an37-1
506,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.000050,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,500,None,{'runtime': 11},udc-an37-1
507,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.000001,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,25,None,{'runtime': 11},udc-an37-1
508,ga_llama3_hparam_search,{},out/,42,torch.bfloat16,0.000001,online,16,0,none,...,50,"[bio-forget-corpus, cyber-forget-corpus]","[wikitext, wikitext]",c4,None,0,100,None,{'runtime': 11},udc-an37-1


In [8]:
all_runs_df.value_counts("host")

host
udc-an37-1    510
Name: count, dtype: int64